### Testing application

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, window, max, min, avg
from pyspark.sql.functions import split
from pyspark.sql.types import StringType, FloatType, StructType, StructField
from pyspark.sql.functions import from_json, col, to_timestamp

In [3]:
# Create a spark session
spark = SparkSession.builder.appName("Twitter").getOrCreate()

In [16]:
# Read kafka stream and subscribe to tweet topic
df = (spark.readStream
          .format('kafka')
          .option('kafka.bootstrap.servers', 'kafka-1:9092')
          .option('startingOffsets', 'latest')
          .option('subscribe', 'twitter')
          .load())

In [9]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
raw_df = (df
         .writeStream
         .queryName("rawdatas")
         .format("memory")
         .start())

In [15]:
raw = spark.sql("select * from rawdatas")
raw.show()

+----+--------------------+-------+---------+------+--------------------+-------------+
| key|               value|  topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------+---------+------+--------------------+-------------+
|null|[7B 22 68 65 6C 6...|twitter|        0|    46|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    47|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    48|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    49|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    50|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    51|2019-10-22 06:07:...|            0|
|null|[7B 22 68 65 6C 6...|twitter|        0|    52|2019-10-22 06:08:...|            0|
+----+--------------------+-------+---------+------+--------------------+-------------+

